In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import random

def ls(self):
    """
    List the contents of the directory.
    Returns:
        List[Path]: A list of Paths in the directory.
    """
    if not self.is_dir():
        raise NotADirectoryError(f"{self} is not a directory")
    
    return list(self.iterdir())
Path.ls = ls

from app.services.audio.audio import AudioSlicer
import io


In [3]:
import asyncio
import io
import json
import logging
from datetime import datetime, timezone

import pandas as pd
from faster_whisper import WhisperModel

In [5]:
model_size = "medium"#large-v3
model = WhisperModel(model_size, device="cuda", compute_type="float16")

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

In [ ]:
audio_path = Path('/home/dima/ssd/1/audio')
path = audio_path.ls()[10]
#path = audio_path/'test.webm'


In [ ]:

audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 120)
slice_duration = audio_slicer.audio.duration_seconds
audio_data = await audio_slicer.export_data()

In [ ]:
audio_slicer.audio

In [ ]:
data = io.BytesIO(audio_data)

In [ ]:
segments, _ = model.transcribe(
                data,
                beam_size=5,
                vad_filter=True,
                word_timestamps=True,
                vad_parameters={"threshold": 0.9},
            )  #
segments = [s for s in list(segments)]

In [6]:
segments_df = pd.DataFrame(segments)

NameError: name 'segments' is not defined

In [1]:
segments_df

NameError: name 'segments_df' is not defined

In [169]:
import io
import torchaudio
import torch
from speechbrain.inference.speaker import EncoderClassifier
from torch.nn.functional import cosine_similarity

# Check if CUDA is available
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda', index=2)

In [170]:

# Load the pre-trained model
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb",run_opts={"device":device})


In [171]:
def encode(audio_data:bytes,device='cuda'):
    signal, fs = torchaudio.load(io.BytesIO(audio_data))
    signal = signal.to(device)

    with torch.no_grad():
        return classifier.encode_batch(signal).squeeze()


In [175]:
results =[]

In [176]:
for n in range(50):
    s1 = segments[n]
    audio_data1, player1 = await slice_audio(path,s1.start,s1.end)
    first = encode(audio_data1)
    
    s2 = segments[n+1]
    audio_data2, player1 = await slice_audio(path,s2.start,s2.end)
    second = encode(audio_data2)
    similarity = cosine_similarity(first, second)
    print(n,similarity)
    results.append(similarity)



0 tensor([0.7730, 0.7733], device='cuda:2')
1 tensor([0.6544, 0.6545], device='cuda:2')
2 tensor([0.6655, 0.6651], device='cuda:2')
3 tensor([0.9394, 0.9395], device='cuda:2')
4 tensor([0.7225, 0.7227], device='cuda:2')
5 tensor([0.5989, 0.5996], device='cuda:2')
6 tensor([0.7621, 0.7625], device='cuda:2')
7 tensor([0.8948, 0.8949], device='cuda:2')
8 tensor([0.6718, 0.6722], device='cuda:2')
9 tensor([0.6608, 0.6610], device='cuda:2')
10 tensor([0.8816, 0.8816], device='cuda:2')
11 tensor([0.3010, 0.3012], device='cuda:2')
12 tensor([0.3543, 0.3558], device='cuda:2')


IndexError: list index out of range

In [13]:
async def slice_audio(path,start,stop):
    audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, start, stop-start)
    slice_duration = audio_slicer.audio.duration_seconds
    audio_data = await audio_slicer.export_data()
    return audio_data, audio_slicer.audio

In [184]:
n=0

In [188]:
audio_data, player = await slice_audio(path,segments[n].start,segments[n].end+10)
player

In [189]:
audio_data, player = await slice_audio(path,segments[n+1].start,segments[n+1].end+10)
player

0 tensor([0.6894, 0.6897], device='cuda:2')
1 tensor([0.6375, 0.6379], device='cuda:2')
2 tensor([0.8876, 0.8876], device='cuda:2')
3 tensor([0.8490, 0.8488], device='cuda:2')
4 tensor([0.7699, 0.7696], device='cuda:2')
5 tensor([0.7871, 0.7872], device='cuda:2')
6 tensor([0.8045, 0.8043], device='cuda:2')
7 tensor([0.7544, 0.7546], device='cuda:2')
8 tensor([0.6912, 0.6914], device='cuda:2')
9 tensor([0.7560, 0.7558], device='cuda:2')
10 tensor([0.7843, 0.7843], device='cuda:2')
11 tensor([0.8191, 0.8190], device='cuda:2')
12 tensor([0.8665, 0.8665], device='cuda:2')
13 tensor([0.6900, 0.6902], device='cuda:2')
14 tensor([0.1986, 0.1986], device='cuda:2')
15 tensor([0.3420, 0.3418], device='cuda:2')
16 tensor([0.7956, 0.7957], device='cuda:2')
17 tensor([0.7502, 0.7503], device='cuda:2')
18 tensor([0.7297, 0.7297], device='cuda:2')
19 tensor([0.7928, 0.7928], device='cuda:2')
20 tensor([0.5627, 0.5627], device='cuda:2')
21 tensor([0.8350, 0.8352], device='cuda:2')
22 tensor([0.6939, 0

In [121]:
results_df = pd.DataFrame([r[0].item() for r in results])

In [122]:
results_df.mean()

0    0.7754
dtype: float64

In [153]:
results_df[results_df<0.7].dropna()

,0
0,0.689421
1,0.637514
8,0.691203
13,0.690042
14,0.198571
15,0.341962
20,0.562745
22,0.693927
23,0.692094
36,0.685714


In [154]:
n=8
audio_data, player = await slice_audio(path,segments[n].start,segments[n].end+2)
player

In [155]:
n+=1
audio_data, player = await slice_audio(path,segments[n].start,segments[n].end+2)
player

In [141]:
segments[n].start

422.24

In [143]:
segments[n].end

423.04

In [144]:
n

71

In [145]:
segments[n]

Segment(id=72, seek=41848, start=422.24, end=423.04, text=' Ну, что-то я не знаю.', tokens=[51765, 7571, 11, 2143, 12, 860, 2552, 1725, 16315, 13, 51863], temperature=0.8, avg_logprob=-0.48556305641351744, compression_ratio=2.078078078078078, no_speech_prob=0.06646728515625, words=[Word(start=422.24, end=422.6, word=' Ну,', probability=0.82861328125), Word(start=422.6, end=422.74, word=' что', probability=0.68896484375), Word(start=422.74, end=422.86, word='-то', probability=0.985595703125), Word(start=422.86, end=422.94, word=' я', probability=0.728515625), Word(start=422.94, end=423.04, word=' не', probability=0.0197906494140625), Word(start=423.04, end=423.04, word=' знаю.', probability=0.41845703125)])

In [79]:
results

[tensor([1.0000, 1.0000], device='cuda:2'),
 tensor([0.6894, 0.6897], device='cuda:2'),
 tensor([0.8360, 0.8360], device='cuda:2'),
 tensor([0.8197, 0.8197], device='cuda:2'),
 tensor([0.7026, 0.7024], device='cuda:2'),
 tensor([0.7714, 0.7713], device='cuda:2'),
 tensor([0.8233, 0.8234], device='cuda:2'),
 tensor([0.7446, 0.7446], device='cuda:2'),
 tensor([0.6541, 0.6543], device='cuda:2'),
 tensor([0.7089, 0.7087], device='cuda:2'),
 tensor([0.7936, 0.7935], device='cuda:2'),
 tensor([0.7077, 0.7076], device='cuda:2'),
 tensor([0.8282, 0.8282], device='cuda:2'),
 tensor([0.7084, 0.7084], device='cuda:2'),
 tensor([0.8232, 0.8234], device='cuda:2'),
 tensor([0.2495, 0.2494], device='cuda:2'),
 tensor([0.7627, 0.7626], device='cuda:2'),
 tensor([0.8023, 0.8025], device='cuda:2'),
 tensor([0.7026, 0.7026], device='cuda:2'),
 tensor([0.5211, 0.5209], device='cuda:2'),
 tensor([0.4283, 0.4281], device='cuda:2'),
 tensor([0.7751, 0.7748], device='cuda:2'),
 tensor([0.7199, 0.7199], device